## Dynamic Programming

+ 1. Overlapping Subproblems   解析问题
+ 2. Overlapping computing saved in a table     保存子问题在表中
+ 3. Parse solution   解析解

In [60]:
from functools import wraps
from collections import defaultdict
from collections import Counter
from functools import lru_cache

## Get the max splitting by enumerate

1、理解函数可作为传参，甚至数组传参

In [2]:
def example(f, arg):
    return f(arg)

In [3]:
def add_ten(num):
    return num + 10

In [4]:
def mul_ten(num):
    return num * 10 

In [5]:
operations = [add_ten, mul_ten]
for f in operations:
    print(example(f, 100))

110
1000


2、理解函数传入函数

In [6]:
called_time = defaultdict(int)

def get_call_times(f):
    result = f()
    print('function: “{}” called once! '.format(f.__name__))
    called_time[f.__name__] += 1
    
    return result

In [7]:
def some_funcion_1(): print('I am function 1')

In [8]:
get_call_times(some_funcion_1)

I am function 1
function: “some_funcion_1” called once! 


In [9]:
called_time

defaultdict(int, {'some_funcion_1': 1})

In [10]:
call_time_with_arg = defaultdict(int)

In [11]:
max(1,2,3,4,5)

5

In [12]:
max([1,2,3])

3

In [13]:
max({1:2,2:2,3:1})

3

In [14]:
def r(n):
    #fname = r.__name__
    #call_time_with_arg[(fname, n)] += 1
    
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [15]:
del call_time_with_arg

3、理解什么是装饰器

In [16]:
from functools import wraps

In [17]:
called_time_with_arg = defaultdict(int)

def get_call_time(f):
    """@param f is a function"""
    @wraps(f)
    def wrap(n):
        """Haha I am warp"""
       # print('I can count')
        result = f(n)
        called_time_with_arg[(f.__name__, n)] += 1
        return result
    return wrap

In [18]:
def add_ten(n): return n + 10

In [19]:
add_ten(10)

20

In [20]:
add_ten = get_call_time(add_ten)

3、理解装饰器带来哪些数据的变化

In [21]:
def add_twenty(n): 
    return n + 20

In [22]:
add_twenty(9)

29

In [23]:
called_time_with_arg

defaultdict(int, {})

In [24]:
def add_twenty(n): 
    return n + 20
add_twenty = get_call_time(add_twenty)
add_twenty(9)

29

In [25]:
called_time_with_arg

defaultdict(int, {('add_twenty', 9): 1})

In [26]:
called_time_with_arg = defaultdict(int)

In [27]:
@get_call_time
def add_twenty(n): 
    return n + 20

In [28]:
add_twenty(9)

29

In [29]:
called_time_with_arg

defaultdict(int, {('add_twenty', 9): 1})

注：出错是因为重复使用了2遍装饰器或没有清空值

4、应用到计算最佳钢材上

In [49]:
from collections import defaultdict
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
price = defaultdict(int)
for i, p in enumerate(original_price): 
    price[i + 1] = p
price[11]

35

In [50]:
called_time_with_arg = defaultdict(int)

In [51]:
solution = {}

In [52]:
def memo(f): 
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed: 
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        
        return result
    
    return _wrap

In [53]:
#@get_call_time
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    # 存了所有的最优结果，在这个函数在不断递归的时候
    solution[n] = (n - max_split, max_split)
    
    return max_price

In [54]:
r(38)

118

In [55]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3),
 21: (11, 10),
 22: (11, 11),
 23: (22, 1),
 24: (22, 2),
 25: (22, 3),
 26: (24, 2),
 27: (25, 2),
 28: (22, 6),
 29: (28, 1),
 30: (28, 2),
 31: (28, 3),
 32: (22, 10),
 33: (22, 11),
 34: (33, 1),
 35: (33, 2),
 36: (33, 3),
 37: (35, 2),
 38: (36, 2)}

In [37]:
# 因为存了最优的结果，所以从结果里，不断从左往右找，直到不能再被划分
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: 
        return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [38]:
r(234)

743

In [39]:
parse_solution(234)

[11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 3]

6、额外计算时间的程序

In [57]:
def func_1(n):
    for i in range(n):
        print(n)

In [58]:
def call_time(func_1, arg): # 脚手架程序 
    start = time.time()
    func_1(arg)
    print('used time: {}'.format(time.time() - start))

6、额外计算时间的程序

##  Edit Distance

In [113]:
solution = {}

In [114]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, '第二个值比第一个值少 {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, '第二个值比第一个值多 {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 
                        '前面是距离值，计算的字符是{}和{},不同的地方在 {} => {}'.format(string1, string2, tail_s1, tail_s2))
        print(both_forward)

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [115]:
edit_distance('123', '12')

(2, '前面是距离值，计算的字符是1和12,不同的地方在 1 => 2')
(2, '前面是距离值，计算的字符是12和1,不同的地方在 2 => 1')
(3, '前面是距离值，计算的字符是123和1,不同的地方在 3 => 1')
(2, '前面是距离值，计算的字符是123和12,不同的地方在 3 => 2')


1

In [116]:
solution

{('1', '1'): '',
 ('1', '12'): '第二个值比第一个值多 2',
 ('12', '1'): '第二个值比第一个值少 2',
 ('12', '12'): '',
 ('123', '1'): '第二个值比第一个值少 3',
 ('123', '12'): '第二个值比第一个值少 3'}

In [117]:
solution = {}
edit_distance('ABCDE', 'ABCCEF')

(2, '前面是距离值，计算的字符是A和AB,不同的地方在 A => B')
(3, '前面是距离值，计算的字符是A和ABC,不同的地方在 A => C')
(4, '前面是距离值，计算的字符是A和ABCC,不同的地方在 A => C')
(5, '前面是距离值，计算的字符是A和ABCCE,不同的地方在 A => E')
(6, '前面是距离值，计算的字符是A和ABCCEF,不同的地方在 A => F')
(2, '前面是距离值，计算的字符是AB和A,不同的地方在 B => A')
(2, '前面是距离值，计算的字符是AB和ABC,不同的地方在 B => C')
(3, '前面是距离值，计算的字符是AB和ABCC,不同的地方在 B => C')
(4, '前面是距离值，计算的字符是AB和ABCCE,不同的地方在 B => E')
(5, '前面是距离值，计算的字符是AB和ABCCEF,不同的地方在 B => F')
(3, '前面是距离值，计算的字符是ABC和A,不同的地方在 C => A')
(2, '前面是距离值，计算的字符是ABC和AB,不同的地方在 C => B')
(3, '前面是距离值，计算的字符是ABC和ABCCE,不同的地方在 C => E')
(4, '前面是距离值，计算的字符是ABC和ABCCEF,不同的地方在 C => F')
(4, '前面是距离值，计算的字符是ABCD和A,不同的地方在 D => A')
(3, '前面是距离值，计算的字符是ABCD和AB,不同的地方在 D => B')
(2, '前面是距离值，计算的字符是ABCD和ABC,不同的地方在 D => C')
(1, '前面是距离值，计算的字符是ABCD和ABCC,不同的地方在 D => C')
(2, '前面是距离值，计算的字符是ABCD和ABCCE,不同的地方在 D => E')
(3, '前面是距离值，计算的字符是ABCD和ABCCEF,不同的地方在 D => F')
(5, '前面是距离值，计算的字符是ABCDE和A,不同的地方在 E => A')
(4, '前面是距离值，计算的字符是ABCDE和AB,不同的地方在 E => B')
(3, '前面是距离值，计算的字符是ABCDE和ABC,不同的地方在 E => C')
(2, '前面是距离值，计算的字符是ABCDE和AB

2